In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp transcript_plots

In [ ]:
#| export
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.sparse import csr_matrix
import anndata as ad
import seaborn as sns
import requests
import sys
import patchworklib as pw
import urllib
import os
import gzip
import shutil
import torch
import matplotlib.pyplot as plt
from scipy.special import gammaln
from scipy.optimize import minimize
from itertools import combinations
from typing import List, Tuple, Dict, Union, Optional
from functools import lru_cache
import pyranges as pr
from pyfaidx import Fasta
import logging

In [ ]:
# | export
class TranscriptData:
    """
    A class for managing transcript and gene information from a GTF file using PyRanges.

    Existing Features:
      - Lookup by transcript ID or gene ID/name
      - Support for exons, CDS, UTR queries
      - Intron coordinate calculation
      - Batch queries
      - Transcript length calculation
      - Caching/memoization for repeated queries
      - Basic logging/error handling

    NEW Features:
      (1) Nucleotide/protein sequence retrieval for CDS (with optional FASTA)
      (2) Alternative splicing analysis with splice junctions, isoform comparisons,
          and junction-chain interpretation.
    """

    def __init__(self, gtf_file: str, reference_fasta: Optional[str] = None):
        """
        Read the GTF file into a PyRanges object and store it.
        Optionally store a path to a reference FASTA for sequence methods.

        Args:
            gtf_file (str): Path to a GTF/GFF file.
            reference_fasta (str, optional): Path to a reference genome FASTA.
        """
        self.gtf_file = gtf_file
        self.reference_fasta = reference_fasta  # store for later use
        logging.info(f"Loading GTF from {gtf_file}. This may take a while...")
        try:
            self.gr = pr.read_gtf(gtf_file)
        except Exception as e:
            logging.error(f"Error reading GTF file: {e}")
            raise
        logging.info("GTF loaded successfully.")

    @lru_cache(maxsize=None)
    def get_exons(self, transcript_id: str) -> pr.PyRanges:
        """
        Returns a PyRanges of exons for the given transcript.
        Results are cached for faster repeat lookups.

        Args:
            transcript_id (str): The transcript ID to filter on.

        Returns:
            pr.PyRanges: PyRanges containing exon features for the transcript.
        """
        exons = self.gr[(self.gr.Feature == "exon") & (self.gr.transcript_id == transcript_id)]
        if len(exons) == 0:
            logging.warning(f"No exons found for transcript {transcript_id}.")
        return exons

    @lru_cache(maxsize=None)
    def get_cds(self, transcript_id: str) -> pr.PyRanges:
        """
        Returns a PyRanges of CDS features for the given transcript.
        Results are cached for faster repeat lookups.

        Args:
            transcript_id (str): The transcript ID to filter on.

        Returns:
            pr.PyRanges: PyRanges containing CDS features for the transcript.
        """
        cds = self.gr[(self.gr.Feature == "CDS") & (self.gr.transcript_id == transcript_id)]
        if len(cds) == 0:
            logging.warning(f"No CDS features found for transcript {transcript_id}.")
        return cds

    @lru_cache(maxsize=None)
    def get_utr(self, transcript_id: str, utr_type: str = None) -> pr.PyRanges:
        """
        Returns a PyRanges of UTR features for the given transcript.
        Optionally specify '5UTR' or '3UTR' to filter further.

        Args:
            transcript_id (str): The transcript ID to filter on.
            utr_type (str, optional): If '5UTR', return only 5' UTR;
                                      if '3UTR', return only 3' UTR;
                                      otherwise return all UTR features.

        Returns:
            pr.PyRanges: PyRanges containing UTR features for the transcript.
        """
        utr = self.gr[(self.gr.Feature.str.contains("UTR", na=False)) & (self.gr.transcript_id == transcript_id)]
        if utr_type == "5UTR":
            utr = utr[utr.Feature == "5UTR"]
        elif utr_type == "3UTR":
            utr = utr[utr.Feature == "3UTR"]

        if len(utr) == 0:
            logging.warning(f"No UTR features found for transcript {transcript_id} (type={utr_type}).")
        return utr

    def get_intron_ranges(self, transcript_id: str) -> pr.PyRanges:
        """
        Compute intron ranges by subtracting exons from the entire transcript range.

        Args:
            transcript_id (str): The transcript ID to filter on.

        Returns:
            pr.PyRanges: PyRanges containing intron coordinates.
        """
        exons = self.get_exons(transcript_id)
        if len(exons) == 0:
            return pr.PyRanges()

        df_exons = exons.df
        chrom = df_exons["Chromosome"].iloc[0]
        strand = df_exons["Strand"].iloc[0]

        start_min = df_exons["Start"].min()
        end_max = df_exons["End"].max()

        transcript_range = pr.PyRanges(
            pd.DataFrame({
                "Chromosome": [chrom],
                "Start": [start_min],
                "End": [end_max],
                "Strand": [strand]
            })
        )
        introns = transcript_range.subtract(exons)
        return introns

    def get_exon_coords_and_strand(self, transcript_id: str) -> Tuple[List[List[int]], Optional[int]]:
        """
        Return exon coordinates and strand (+1 or -1) for a given transcript ID,
        mimicking the style of the Ensembl API example (list of [end, start] pairs).

        Args:
            transcript_id (str): The transcript ID to query.

        Returns:
            (exon_coord, strand):
                exon_coord is a list of [end, start] pairs
                strand is +1 or -1
        """
        exons = self.get_exons(transcript_id)
        if len(exons) == 0:
            return ([], None)

        df = exons.df.sort_values(by="Start")
        strand_symbol = df["Strand"].iloc[0]  # '+' or '-'
        strand = 1 if strand_symbol == '+' else -1

        exon_coord = [[row.End, row.Start] for _, row in df.iterrows()]
        return (exon_coord, strand)

    def get_transcript_length(self, transcript_id: str) -> int:
        """
        Return the total length of exons for the given transcript.

        Args:
            transcript_id (str): The transcript ID to query.

        Returns:
            int: Sum of all exon lengths for this transcript.
        """
        exons = self.get_exons(transcript_id)
        if len(exons) == 0:
            return 0
        df = exons.df
        lengths = df["End"] - df["Start"]
        return lengths.sum()

    def get_chromosome(self, transcript_id: str) -> Optional[str]:
        """
        Return the chromosome/contig name for the given transcript.
        Assumes that all exons in this transcript are on the same chromosome.

        Args:
            transcript_id (str): The transcript ID to query.

        Returns:
            str or None: Chromosome name (e.g., 'chr1', '1', etc.) or None if not found.
        """
        exons = self.get_exons(transcript_id)
        if len(exons) == 0:
            return None
        df = exons.df
        return df["Chromosome"].iloc[0]

    def get_strand(self, transcript_id: str) -> Optional[int]:
        """
        Return +1 or -1 for the transcript's strand.

        Args:
            transcript_id (str): The transcript ID to query.

        Returns:
            int or None: 1 or -1, or None if not found.
        """
        exons = self.get_exons(transcript_id)
        if len(exons) == 0:
            return None
        df = exons.df
        strand_symbol = df["Strand"].iloc[0]
        return 1 if strand_symbol == '+' else -1

    def get_transcripts_by_gene_id(self, gene_id: str) -> List[str]:
        """
        Return a list of transcript IDs associated with a given gene_id.

        Args:
            gene_id (str): The gene ID to search for.

        Returns:
            List[str]: Transcript IDs for that gene.
        """
        df = self.gr.df
        subset = df[df.gene_id == gene_id]
        t_ids = subset["transcript_id"].dropna().unique()
        if len(t_ids) == 0:
            logging.warning(f"No transcripts found for gene ID {gene_id}.")
        return list(t_ids)

    def get_transcripts_by_gene_name(self, gene_name: str) -> List[str]:
        """
        Return a list of transcript IDs associated with a given gene_name.

        Args:
            gene_name (str): The gene name to search for (e.g. BRCA1).

        Returns:
            List[str]: Transcript IDs for that gene.
        """
        df = self.gr.df
        if "gene_name" not in df.columns:
            logging.warning("No 'gene_name' column in GTF; cannot filter by gene name.")
            return []
        subset = df[df.gene_name == gene_name]
        t_ids = subset["transcript_id"].dropna().unique()
        if len(t_ids) == 0:
            logging.warning(f"No transcripts found for gene name {gene_name}.")
        return list(t_ids)

    def get_exons_batch(self, transcript_ids: List[str]) -> Dict[str, pr.PyRanges]:
        """
        Return a dict of transcript_id -> exons PyRanges for a list of transcript IDs.
        Useful for batch queries.

        Args:
            transcript_ids (list of str): List of transcript IDs to fetch.

        Returns:
            dict: {transcript_id: PyRanges}
        """
        result = {}
        for tid in transcript_ids:
            result[tid] = self.get_exons(tid)
        return result

    def get_exon_coords_and_strand_batch(self, transcript_ids: List[str]) -> Dict[str, Tuple[List[List[int]], Optional[int]]]:
        """
        Return a dict of transcript_id -> (exon_coord, strand), for batch querying.

        Args:
            transcript_ids (list of str): Transcript IDs to fetch.

        Returns:
            dict: {transcript_id: ([ [end, start], ... ], strand) }
        """
        result = {}
        for tid in transcript_ids:
            result[tid] = self.get_exon_coords_and_strand(tid)
        return result

    # ---------------------------------------------------------------------
    # NEW FEATURE (1): Sequence Extraction
    # ---------------------------------------------------------------------

    def get_cds_sequence(self,
                         transcript_id: str,
                         reference_fasta: Optional[str] = None
                         ) -> Optional[str]:
        """
        Return the nucleotide sequence of the CDS for a given transcript.
        If reference_fasta is not provided, the method will use self.reference_fasta,
        or prompt the user if that is also None.

        Requires pyfaidx and a valid reference FASTA.

        Args:
            transcript_id (str): The transcript ID to query.
            reference_fasta (str, optional): Path to the reference genome FASTA file.

        Returns:
            str or None: Nucleotide sequence of the CDS, or None if no CDS found.
        """
        if Fasta is None:
            logging.error("pyfaidx is not installed. Cannot extract sequences.")
            return None

        # If no explicit FASTA was passed, fallback to the instance-level FASTA
        if reference_fasta is None:
            if self.reference_fasta:
                reference_fasta = self.reference_fasta
            else:
                reference_fasta = input("Please provide path to the reference genome FASTA file: ")

        cds = self.get_cds(transcript_id)
        if len(cds) == 0:
            logging.warning(f"No CDS found for transcript {transcript_id}.")
            return None

        # Open reference FASTA
        fa = Fasta(reference_fasta)

        # Sort CDS features by genomic position (important if multiple exons)
        df = cds.df.sort_values(by="Start")

        # Extract sequence pieces and concatenate
        seq_pieces = []
        for _, row in df.iterrows():
            chrom = row["Chromosome"]
            start = row["Start"]
            end = row["End"]
            strand = row["Strand"]

            piece = fa[chrom][start:end].seq
            if strand == "-":
                # Reverse complement if negative strand
                piece = self._revcomp(piece)
            seq_pieces.append(piece)

        return "".join(seq_pieces)

    def get_protein_sequence(self,
                             transcript_id: str,
                             reference_fasta: Optional[str] = None
                             ) -> Optional[str]:
        """
        Return the translated protein sequence (in one-letter code) for a given transcript’s CDS.
        If reference_fasta is not provided, the method will use self.reference_fasta,
        or prompt the user if that is also None.

        Requires pyfaidx and a valid reference FASTA.

        Args:
            transcript_id (str): The transcript ID to query.
            reference_fasta (str, optional): Path to the reference genome FASTA file.

        Returns:
            str or None: Amino acid sequence, or None if no CDS is found.
        """
        cds_seq = self.get_cds_sequence(transcript_id, reference_fasta)
        if cds_seq is None:
            return None

        codon_table = self._get_standard_codon_table()
        protein = []
        for i in range(0, len(cds_seq), 3):
            codon = cds_seq[i : i + 3]
            if len(codon) < 3:
                break  # incomplete codon
            aa = codon_table.get(codon, "X")  # unknown => 'X'
            if aa == "*":  # stop codon
                break
            protein.append(aa)
        return "".join(protein)

    def _revcomp(self, seq: str) -> str:
        """
        Return the reverse-complement of a nucleotide sequence.
        """
        complement = {
            "A": "T", "C": "G", "G": "C", "T": "A",
            "a": "t", "c": "g", "g": "c", "t": "a",
            "N": "N", "n": "n"
        }
        rev = []
        for base in reversed(seq):
            rev.append(complement.get(base, "N"))
        return "".join(rev)

    def _get_standard_codon_table(self) -> Dict[str, str]:
        """
        Return a minimal codon table mapping triplets to single-letter amino acids.
        Stop codon => '*'
        """
        return {
            "ATA":"I","ATC":"I","ATT":"I","ATG":"M","ACA":"T","ACC":"T","ACG":"T","ACT":"T",
            "AAC":"N","AAT":"N","AAA":"K","AAG":"K","AGC":"S","AGT":"S","AGA":"R","AGG":"R",
            "CTA":"L","CTC":"L","CTG":"L","CTT":"L","CCA":"P","CCC":"P","CCG":"P","CCT":"P",
            "CAC":"H","CAT":"H","CAA":"Q","CAG":"Q","CGA":"R","CGC":"R","CGG":"R","CGT":"R",
            "GTA":"V","GTC":"V","GTG":"V","GTT":"V","GCA":"A","GCC":"A","GCG":"A","GCT":"A",
            "GAC":"D","GAT":"D","GAA":"E","GAG":"E","GGA":"G","GGC":"G","GGG":"G","GGT":"G",
            "TCA":"S","TCC":"S","TCG":"S","TCT":"S","TTC":"F","TTT":"F","TTA":"L","TTG":"L",
            "TAC":"Y","TAT":"Y","TAA":"*","TAG":"*","TGC":"C","TGT":"C","TGA":"*","TGG":"W"
        }

    # ---------------------------------------------------------------------
    # NEW FEATURE (2): Alternative Splicing Analysis
    # ---------------------------------------------------------------------

    def get_splice_junctions(self, transcript_id: str) -> List[Tuple[int, int]]:
        """
        Return the genomic start/end positions for each splice junction
        (the exon-exon boundaries) for a given transcript.

        Args:
            transcript_id (str): The transcript ID to query.

        Returns:
            List[Tuple[int, int]]: List of (donor_site, acceptor_site) for each splice junction.
        """
        exons = self.get_exons(transcript_id)
        if len(exons) < 2:
            logging.info(f"Transcript {transcript_id} has fewer than 2 exons; no internal junctions.")
            return []

        df_exons = exons.df.sort_values(by="Start")
        junctions = []
        for i in range(len(df_exons) - 1):
            exon_end = df_exons.iloc[i]["End"]
            next_exon_start = df_exons.iloc[i + 1]["Start"]
            junctions.append((exon_end, next_exon_start))

        return junctions

    def compare_transcripts_across_gene(self, gene_id: str) -> pd.DataFrame:
        """
        Compare exons of all transcripts for a given gene.
        Returns a DataFrame of all exons grouped by transcript ID,
        so you can quickly see which exons are shared or unique across isoforms.

        Args:
            gene_id (str): The gene ID to compare.

        Returns:
            pd.DataFrame: A dataframe with columns
                          [transcript_id, Chromosome, Start, End, Strand].
        """
        tid_list = self.get_transcripts_by_gene_id(gene_id)
        if not tid_list:
            logging.warning(f"No transcripts found for gene {gene_id}.")
            return pd.DataFrame()

        all_exons = []
        for tid in tid_list:
            exons = self.get_exons(tid)
            if len(exons) == 0:
                continue
            df_exons = exons.df.copy()
            df_exons["transcript_id"] = tid
            all_exons.append(
                df_exons[["transcript_id", "Chromosome", "Start", "End", "Strand"]]
            )

        if not all_exons:
            return pd.DataFrame()

        result = pd.concat(all_exons, ignore_index=True)
        result.sort_values(by=["transcript_id", "Start"], inplace=True)
        return result

    # ---------------------------------------------------------------------
    # Junction Chain Interpretation
    # ---------------------------------------------------------------------

    def get_junction_chain_signature(self, transcript_id: str) -> Optional[Tuple[Tuple[int, int], ...]]:
        """
        Return a tuple of (exon_end, next_exon_start) pairs for each splice junction
        in the given transcript. This provides a 'signature' to compare across
        transcripts to see if they have the same junction chain.

        Args:
            transcript_id (str): The transcript ID to query.

        Returns:
            tuple of (int, int) or None:
                A tuple of (end_of_exon_i, start_of_exon_(i+1)) for i in [0..n_exons-2].
                Returns None if fewer than 2 exons (no internal junctions).
        """
        exons = self.get_exons(transcript_id)
        if len(exons) < 2:
            logging.info(f"Transcript {transcript_id} has fewer than 2 exons; no junction chain.")
            return None

        df_exons = exons.df.sort_values(by="Start")
        junctions = []
        for i in range(len(df_exons) - 1):
            exon_end = df_exons.iloc[i]["End"]
            next_exon_start = df_exons.iloc[i + 1]["Start"]
            junctions.append((exon_end, next_exon_start))

        return tuple(junctions)

    def interpret_unique_junction_chains(self, gene_id: str) -> pd.DataFrame:
        """
        Group transcripts of a given gene by their unique junction chain signatures.
        This helps identify which isoforms share the exact same exon-exon boundaries
        and which are unique.

        Args:
            gene_id (str): The gene ID to analyze.

        Returns:
            pd.DataFrame:
                Columns:
                  - 'junction_chain_signature': The tuple of (exon_end, next_exon_start) pairs.
                  - 'transcript_count': How many transcripts share this chain.
                  - 'transcripts': A list of transcript IDs that have this chain.
        """
        tid_list = self.get_transcripts_by_gene_id(gene_id)
        if not tid_list:
            logging.warning(f"No transcripts found for gene {gene_id}.")
            return pd.DataFrame()

        chain_map = {}  # {chain_signature: [transcript_1, transcript_2, ...]}

        for tid in tid_list:
            signature = self.get_junction_chain_signature(tid)
            if signature is None:  # e.g., single-exon transcripts
                continue
            chain_map.setdefault(signature, []).append(tid)

        rows = []
        for chain_sig, transcripts in chain_map.items():
            rows.append({
                "junction_chain_signature": chain_sig,
                "transcript_count": len(transcripts),
                "transcripts": transcripts
            })

        df = pd.DataFrame(rows)
        df.sort_values("transcript_count", ascending=False, inplace=True)
        df.reset_index(drop=True, inplace=True)
        return df

In [ ]:
# | export
class TranscriptPlots:
    def __init__(self, gtf_file=None, reference_fasta=None):
        self.transcript_data = None
        if gtf_file is not None:
            self.transcript_data = TranscriptData(gtf_file=gtf_file, reference_fasta=reference_fasta)

    def get_transcript_info(self, transcript_id):
        if self.transcript_data is None:
            raise NotImplementedError()

        data_dict = dict()
        keys = ['transcript_id', 'transcript_name', 'transcript_type', 'cds_start', 'cds_end', 'chromosome', 'strand']
        for key in keys:
            data_dict[key] = 0
        data_dict['cds_end'] = -sys.maxsize
        data_dict['cds_start'] = sys.maxsize
        cds = self.transcript_data.get_cds(transcript_id)
        df = cds.df.sort_values(by="Start")
        for _, row in df.iterrows():
            data_dict['transcript_id'] = row["transcript_id"]
            data_dict['transcript_name'] = row["transcript_name"]
            data_dict['transcript_type'] = row["transcript_type"]
            data_dict['cds_start'] = min(row["Start"], data_dict['cds_start'])
            data_dict['cds_end'] = max(row["Start"], data_dict['cds_end'])
            data_dict['chromosome'] = row["Chromosome"]
            data_dict['strand'] = row["Strand"]

        return data_dict

    def _get_coord_from_tscrpt_id(self, transcript_id):
        if self.transcript_data is None:
            if '.' in transcript_id:
                transcript_id = transcript_id.split('.')[0]
            server = "https://rest.ensembl.org"
            ext = "/lookup/id/" + transcript_id + "?expand=1"

            r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

            if not r.ok:
                r.raise_for_status()
                sys.exit()

            decoded = r.json()
            exon_list = list(decoded['Exon'])
            exon_coord = []
            for i, e in enumerate(exon_list):
                coord = [e.get('end'), e.get('start')]
                exon_coord.append(coord)
            strand = decoded['strand']
            return(exon_coord, strand)
        else:
            return self.transcript_data.get_exon_coords_and_strand(transcript_id)

    def _draw_transcript(self, exons, direction, color, transcript_name, offset=0, start_override=None, end_override=None, no_render=False):
        if not no_render:
            plt.axes()
            plt.xlim((-0.1, 1))
            plt.ylim((-0.3, 0.3))
            plt.margins(0.2)
            plt.axis('off')
            fig = plt.gcf()
            fig.set_size_inches(20, 2)
        height = 0.2
        plt.plot([offset + 0.1, offset + 0.1], linestyle='solid', linewidth=0.5, c='grey')
        j = 0
        k = 1
        if direction == 1:
            pos_start = exons[0][1]
            pos_end = exons[-1][0]
        else: #direction == -1
            pos_start = exons[-1][1]
            pos_end = exons[0][0]
            j = 1
            k = 0
        real_start = pos_start
        real_end = pos_end
        if start_override is not None and end_override is not None:
            pos_start = start_override
            pos_end = end_override
        total_length = pos_end - pos_start
        total_length_with_margin = 1.05 * total_length
        pos_start_with_margin = pos_start - 0.025*total_length
        for i, exon in enumerate(exons):
            rectangle = plt.Rectangle(((exon[j] - pos_start_with_margin)/total_length_with_margin,offset), (exon[k] - exon[j])/total_length_with_margin, height, fc=color,ec="black")
            plt.gca().add_patch(rectangle)
        if i > 0:
            arrow = None
            if direction < 0:
                arrow = plt.arrow(1, offset - height/4, -1, 0, width=0.0015, head_length=0.01, head_width=0.1, length_includes_head=True, overhang=1)
            else:
                arrow = plt.arrow(0, offset - height/4, 1, 0, width=0.0015, head_length=0.01, head_width=0.1, length_includes_head=True, overhang=1)
            plt.gca().add_patch(arrow)
        plt.plot(np.array([0.025 + (real_start - pos_start) / (total_length)/1.05, 0.025 + (real_start - pos_start) / (total_length)/1.05]), np.array([offset - height/4 - 0.03, offset - height/4 + 0.03]), color='black')
        plt.plot(np.array([1 - 0.025 - (pos_end - real_end) / (total_length)/1.05, 1 - 0.025 - (pos_end - real_end) / (total_length)/1.05]), np.array([offset - height/4 - 0.03, offset - height/4 + 0.03]), color='black')
        plt.text(0.025 + (real_start - pos_start) / (total_length)/1.05, offset - height/4 - 0.075, real_start, horizontalalignment='center', verticalalignment='center', fontsize=9)
        plt.text(1 - 0.025 - (pos_end - real_end) / (total_length)/1.05, offset - height/4 - 0.075, real_end, horizontalalignment='center', verticalalignment='center', fontsize=9)
        plt.text(1, offset - height, transcript_name, horizontalalignment='right', verticalalignment='top', fontsize=12)
        if not no_render:
            plt.show()

    def _draw_transcripts_list(self, trs_to_show, _ax, colors=None):
        transcripts_id = trs_to_show
        exons = []
        directions = []
        for tr in transcripts_id:
            t, d = self._get_coord_from_tscrpt_id(tr)
            exons += [t]
            directions += [d]
        if colors is None:
            colors = []
            for i in range(len(exons)):
                colors.append(self.colors[i % len(self.colors)])
        def get_limits(ex, dir):
            start = sys.maxsize
            end = -sys.maxsize
            for (e, d) in zip(ex, dir):
                if d == 1:
                    start = min(start,e[0][1])
                    end = max(end, e[-1][0])
                else:
                    start = min(start,e[-1][1])
                    end = max(end, e[0][0])
            return (start, end)
        plt.axes()
        plt.xlim((-0.1, 1.1))
        plt.ylim((0.1 - 0.5 *  len(exons), 0.3))
        plt.margins(0.2)
        plt.axis('off')
        fig = plt.gcf()
        fig.set_size_inches(20, len(exons) * 2)
        i = 0
        (start, end) = get_limits(exons, directions)
        for (ex, di, co, name) in zip(exons, directions, colors, transcripts_id):
            self._draw_transcript(ex, di, co, name, offset= -0.5 * i, start_override=start, end_override=end, no_render=True)
            i+=1
        if _ax is None:
            plt.show()
        else:
            return plt

    def draw_transcripts_list(self, transcripts_ids, colors=None):
        self._draw_transcripts_list(transcripts_ids, None, colors)

    def draw_transcripts_list_unscaled(self, transcripts_id, colors=None):
        exons = []
        directions = []
        for tr in transcripts_id:
            t, d = self._get_coord_from_tscrpt_id(tr)
            exons += [t]
            directions += [d]
        if colors is None:
            colors = []
            for i in range(len(exons)):
                colors.append(self.colors[i % len(self.colors)])
        def get_limits(ex, dir):
            start = sys.maxsize
            end = -sys.maxsize
            for (e, d) in zip(ex, dir):
                if d == 1:
                    start = min(start,e[0][1])
                    end = max(end, e[-1][0])
                else:
                    start = min(start,e[-1][1])
                    end = max(end, e[0][0])
            return (start, end)

        def move_exons(exons_list, direction, max_width, min_width):
            i = 1
            if direction == 1:
                while i < len(exons_list):
                    if exons_list[i][1] - exons_list[i - 1][0] < min_width:
                        diff = min_width - (exons_list[i][1] - exons_list[i - 1][0])
                        exons_list[i][0] += diff
                        exons_list[i][1] += diff
                    elif exons_list[i][1] - exons_list[i - 1][0] > max_width:
                        diff = (exons_list[i][1] - exons_list[i - 1][0]) - max_width
                        exons_list[i][0] -= diff
                        exons_list[i][1] -= diff
                    i += 1
            else:
                while i < len(exons_list):
                    if exons_list[i][0] - exons_list[i - 1][1] < min_width:
                        diff = min_width - (exons_list[i][0] - exons_list[i - 1][1])
                        exons_list[i][1] += diff
                        exons_list[i][0] += diff
                    elif exons_list[i][0] - exons_list[i - 1][1] > max_width:
                        diff = (exons_list[i][0] - exons_list[i - 1][1]) - max_width
                        exons_list[i][1] -= diff
                        exons_list[i][0] -= diff
                    i += 1
            return exons_list
        plt.axes()
        plt.xlim((-0.1, 1.1))
        plt.ylim((0.1 - 0.5 *  len(exons), 0.3))
        plt.margins(0.2)
        plt.axis('off')
        fig = plt.gcf()
        fig.set_size_inches(20, len(exons) * 2)
        i = 0
        fake_exons = []
        fake_se = []
        for exon_list, direction in zip(exons, directions):
            fake_exon = []
            (start, end) = get_limits([exon_list], [direction])
            full_length = end - start
            max_width = full_length / (len(exon_list) - 2 if len(exon_list) > 2 else 1)
            min_width = full_length / (10 * (len(exon_list) + 5))
            max_width_e = full_length / 2
            min_width_e = full_length / 20
            for exon in exon_list:
                fake_exon_start = 0
                fake_exon_end = 0
                if direction == 1:
                    fake_exon_start = exon[1]
                    if (exon[0] - exon[1]) > max_width:
                        fake_exon_end = exon[1] + max_width
                    elif (exon[0] - exon[1]) < min_width:
                        fake_exon_end = exon[1] + min_width
                    else:
                        fake_exon_end = exon[0]
                    fake_exon.append([fake_exon_end, fake_exon_start])
                else:
                    fake_exon_start = exon[0]
                    if (exon[1] - exon[0]) > max_width:
                        fake_exon_end = exon[0] + max_width
                    elif (exon[1] - exon[0]) < min_width:
                        fake_exon_end = exon[0] + min_width
                    else:
                        fake_exon_end = exon[1]
                    fake_exon.append([fake_exon_start, fake_exon_end])
            fake_exon = move_exons(fake_exon, direction, max_width_e, min_width_e)
            fake_exons.append(fake_exon)
            (fake_s, fake_e) = get_limits([fake_exon], [direction])
            fake_se.append([fake_s, fake_e])
        for (ex, di, co, name) in zip(fake_exons, directions, colors, transcripts_id):
            self._draw_transcript(ex, di, co, name, offset= -0.5 * i, start_override=fake_se[i][0], end_override=fake_se[i][1], no_render=True)
            i+=1
        plt.show()

